![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/DEID_PHI_TEXT_MULTI.ipynb)

# **Usando Modelos de Desidentificação**

Esse notebook demonstra o uso de modelos de desidentificação em documentos clínicos com uso das bibliotecas `Spark NLP` e `Healthcare NLP`.

## **Setup (Colab)**

Devemos enviar o nosso arquivo de acesso à biblioteca `Healthcare NLP`.

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs

Instalamos os pacotes necessários:

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Importamos os módulos e classes que vamos utilizar.

In [ ]:
from johnsnowlabs import nlp, medical

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.settings.enforce_versions=True
nlp.install(refresh_install=True)

In [ ]:
from johnsnowlabs import nlp, medical
# Automatically load license data and start a session with all jars user has access to

spark = nlp.start()

In [ ]:
spark

In [ ]:
from pyspark.sql import functions as F

import pandas as pd
pd.set_option('display.max_colwidth', 200)

## **Exemplo: Texto Clínico**

In [ ]:
text_portuguese = ["""Dados do paciente.
Nome: Mauro.
Apelido: Gonçalves.
NISS: 26 63514095.
Endereço: Calle Miguel Benitez 90.
CÓDIGO POSTAL: 28016.
Dados de cuidados.
Data de nascimento: 03/03/1946.
País: Portugal.
Idade: 70 anos Sexo: M.
Data de admissão: 12/12/2016.
Doutor: Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Relatório clínico do paciente: Paciente de 70 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.
Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.
O exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV.
A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.
Hemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicéridos de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml.
A citologia da urina era repetidamente desconfiada por malignidade.
A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis.
A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso.
O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.
A cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga.
Referido por: Miguel Santos - Avenida dos Aliados, 22 Portugal E-mail: nnavcu@hotmail.com.
"""]

# **Construindo o Pipeline para processar o texto**

Os Pipelines pré-treinados são capazes de processar textos (ou listas de textos) diretamente com o método `annotate`.

Os pipelines de desidentificação são capazes de fazer as seguintes formas de anonimização:

- Usando a entidade nomeada para substituir a menção
- Usando caracteres (por exemple, asterisco `*`) para substituir a menções (mantendo o mesmo número de caracteres)
- Usando caracteres, mas com número fixo
- Usando anonimação por substituição aleatória de menções ("obfuscação", como troca de nomes, endereços, etc.)

In [ ]:
deid_pipeline = nlp.PretrainedPipeline("clinical_deidentification", "pt", "clinical/models")
deid_res = deid_pipeline.annotate(text_portuguese)[0]
df= pd.DataFrame(
    list(zip(
        deid_res["sentence"], deid_res["masked"],
        deid_res["masked_with_chars"],
        deid_res["masked_fixed_length_chars"],
        deid_res["obfuscated"]
    )),
    columns= ["Frase", "Anonimizado com Entidade", "Anonimizado com Caractere", "Anonimizado com Número Fixo de Caractere", "Anonimizado com substituição de Nomes"]
)

print("\n", f"Resultado do texto desidentificado: ")

display(df.head())

clinical_deidentification download started this may take some time.
Approx size to download 1.2 GB
[OK!]

 Resultado do texto desidentificado: 


,Frase,Anonimizado com Entidade,Anonimizado com Caractere,Anonimizado com Número Fixo de Caractere,Anonimizado com substituição de Nomes
0,Dados do paciente.,Dados do <SEX>.,Dados do [******].,Dados do ****.,Dados do homen.
1,Nome: Mauro.,Nome: <PATIENT>.,Nome: [***].,Nome: ****.,Nome: Josefina Batista.
2,Apelido: Gonçalves.,Apelido: <PATIENT>.,Apelido: [*******].,Apelido: ****.,Apelido: Tiago Santos.
3,NISS: 26 63514095.,NISS: <PHONE> 63514095.,NISS: ** 63514095.,NISS: **** 63514095.,NISS: 75 63514095.
4,Endereço: Calle Miguel Benitez 90.\nCÓDIGO POSTAL: 28016.,Endereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>.,Endereço: [*********************].\nCÓDIGO POSTAL: [***].,Endereço: ****.\nCÓDIGO POSTAL: ****.,"Endereço: Avenida Da Liberdade, 17.\nCÓDIGO POSTAL: 79305."


## Usando o modelo de Identificação de Entidades Nomeadas

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.WordEmbeddingsModel.pretrained("w2v_cc_300d", "pt")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("word_embeddings")

entidades_simplificado = medical.NerModel.pretrained("ner_deid_generic", "pt", "clinical/models")\
    .setInputCols(["sentence","token","word_embeddings"])\
    .setOutputCol("ner_simplificado")

entidades_detalhe = medical.NerModel.pretrained("ner_deid_subentity", "pt", "clinical/models")\
    .setInputCols(["sentence","token","word_embeddings"])\
    .setOutputCol("ner_detalhe")

ner_converter_simplificado = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_simplificado"])\
    .setOutputCol("ner_chunk_simplificado")

ner_converter_detalhe = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_detalhe"])\
    .setOutputCol("ner_chunk_detalhe")

nlpPipeline = nlp.Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer,
    embeddings,
    entidades_simplificado,
    entidades_detalhe,
    ner_converter_simplificado,
    ner_converter_detalhe
])

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
w2v_cc_300d download started this may take some time.
Approximate size to download 1.1 GB
[OK!]
ner_deid_generic download started this may take some time.
Approximate size to download 14.3 MB
[OK!]
ner_deid_subentity download started this may take some time.
Approximate size to download 14.3 MB
[OK!]


In [ ]:
spark_df = spark.createDataFrame(pd.DataFrame({"text": text_portuguese})).toDF("text")

In [ ]:
# Envia o texto para Data Frame do Spark
spark_df = spark.createDataFrame([text_portuguese]).toDF("text")

result_df = nlpPipeline.fit(spark_df).transform(spark_df)

result_df.selectExpr(
    "ner_chunk_simplificado as ner_simplificado",
    "ner_chunk_detalhe as ner_detalhe"
).show(truncate=200)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                        ner_simplificado|                                                                                                                                                                                             ner_detalhe|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
exploded_df = result_df.select(
    F.explode(
        F.arrays_zip(
            F.expr("ner_chunk_simplificado.result as chunk_simplificado"),
            F.expr("ner_chunk_simplificado.metadata as metadata_simplificado"),
            F.expr("ner_chunk_detalhe.result as chunk_detalhe"),
            F.expr("ner_chunk_detalhe.metadata as metadata_detalhe"),
        )
    ).alias("cols")
).select(
    F.expr("cols.chunk_simplificado"),
    F.expr("cols.metadata_simplificado.entity").alias("entidade_simplificado"),
    F.expr("cols.chunk_detalhe"),
    F.expr("cols.metadata_detalhe.entity").alias("entidade_detalhe")
).toPandas()

exploded_df.head()

,chunk_simplificado,entidade_simplificado,chunk_detalhe,entidade_detalhe
0,Mauro,NAME,Mauro,PATIENT
1,Gonçalves,NAME,Gonçalves,PATIENT
2,26 63514095,ID,26 63514095,ID
3,Miguel Benitez 90,LOCATION,Calle Miguel Benitez 90,STREET
4,28016,LOCATION,28016,ZIP


In [ ]:
exploded_df.shape

(14, 4)

In [ ]:
exploded_df

,chunk_simplificado,entidade_simplificado,chunk_detalhe,entidade_detalhe
0,Mauro,NAME,Mauro,PATIENT
1,Gonçalves,NAME,Gonçalves,PATIENT
2,26 63514095,ID,26 63514095,ID
3,Miguel Benitez 90,LOCATION,Calle Miguel Benitez 90,STREET
4,28016,LOCATION,28016,ZIP
5,03/03/1946,DATE,03/03/1946,DATE
6,70,AGE,Portugal,COUNTRY
7,12/12/2016,DATE,70,AGE
8,Ignacio Navarro,NAME,12/12/2016,DATE
9,28 28 70973,CONTACT,Ignacio Navarro,DOCTOR


## Mais exemplos

In [ ]:
textos = [
    ['ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNIA INGUINAL Conduta:EXAMES \u200b'],
    ['EVOLUÇÃO DE ENFERMAGEM 6 º DIH Hipótese Diagnóstica: K35 Apendicite aguda Evolução (estado geral): 05/09/2023 09:30 Faço contato com Dr Marcelo Fiusa via telefone informando que paciente esta com cefaleia intensa medicado conforme prescrição sem melhora ,o mesmo orienta avaliação do anestesista e faz o contato.\u200b10:16 Paciente ainda mantem cefaleia aprazo paracetamol 40 gts,aguarda avaliação do anestesista Sinais Vitais: PA: 110 x 60 / FC: 84 / SATO2: 98 / FR: 20 / Temperatura: 36,8 / Dor: Categórica numérica / 0 / Peso: 55 / Altura: 1,59 Escalas: Braden: / Maddox: Meta do dia: Comunicação com o paciente e/ou cuidador: _____________________________MYLLENA STEPHANIE DOS SANTOS DE PAULA \u200b'],
    ['\u200bPSA\u200b\u200bPACIENTE REFERE LESÃO PRURIGINOSA HA 2 DIAS EM DORSO, AXILA E REGIAO EPIGASTRICA, HOJE ASSOCIADAS A EDEMA PERIORBITAL. \u200b\u200bEXAME FISICO\u200bBEG CHAAAE LOTE \u200bNEURO: ECG 15, PIFR \u200bPRESENÇA DE LESOES PRURIGINOSAS EM REGIAO DO DORSO\u200b\u200bHD: REAÇÃO ALERGICA\u200b\u200bCONDUTA:\u200bHIDROCORTISONA EV \u200bSINTOMATICOS ORIENTO SINAIS DE ALARME E RETORNO, SN \u200bALTA APÓS '],
    ['​\u200bCONSULTA MÉDICA CASE (CMR)\u200b\u200b Retorno Repouso\u200b\u200bDIAGNÓSTICOS: \u200b-SINDROME DEMENCIAL PROVAVEL VASCULAR CDR 2\u200b>>GTT 12/2022-TROCA EM 07/2023\u200b>>LPP SACRAL GRAU 3- CICATRIZADA\u200b-O2 ESPORADICO POS BANHO-DAC COM RM 2005 + HAS + ICC-DRC NAO DIALITICA-DMNID-COVID 09/2022-INTERNAÇOES RECORRENTES POR BCP ASPIRATIVA\u200b-ILPI DESDE 07/2022 \u200bEVOLUÇÃO CLÍNICA ATUAL::\u200bEsposa do paciente vem sem o mesmo. Refere que ha 15 dias iniciou com rinorreia anterior e tosse, foi diagnosticado com broncopneumonia e prescrito bactrim sem melhora, no momento esta em uso de norfloxacino. Paciente morador de ILPI esposa nao sabe falar se esta melhor. Mantido dieta via gtt exclusiva pois perdeu peso no periodo, com 300ml 5x/dia.Traz exames recentes; EXAMES ANTERIORES:\u200b- 09/2023-\u200bHB; 14,1/ LEUCO: 4030/ PLAQ: 131000/ UR: 62/ CR; 1,3/ GLICOSE: 112/ NA: 124/ K: 5,1/ PCR: 1,7/ VLDL: 10/ AC URICO: 7,1/ CALCIO: 8,8/ MAG: 1,9/ TGP: 20/ GGT: 29/ FA: 104/ CPK: 35/ PSAT: 0,64/ PSAL: 0,08/ TSH: 2,07/ VITD: 40/ URINA SEM ALTERAÇOES/ ALBUMINA: 3,8/ VITB12: 903/ CT: 68/ HDL: 37/ LDL: 18/ TGO: 29/ TRIG: 52/ GLICADA: 5,901/2023- PANGASTRITE MODERADA COM H PYLORI NEGATIVO\u200b-LAB 01/2023 SEM ALTERAÇOES\u200b-PSAP; 39 COM REFLUXO TRICUSPIDE-RITMO IRREGULAR. SEM DEMAIS QUEIXAS\u200b06/07: HB 12,3 HT 37,4 LEUCO 7500 SEG 90% PLAQ 38 MIL CREAT 0,8 PCR 12 K 3 SODIO 150. UREIA 60 08/07: Hb 11,5, Leuco 4800, Plt 35.000, Cr 1,1, K 3,6, Na 147, U 82.12/07: HB 12,8 HT 38,1 LEUCO 3900 SEG 56% LINF 33% PLAQ 65 MIL CREAT 1,4 PCR 5,04 K 4,6 SODIO 142 UREIA 84 15/07/22: HB 12,7, HT 36,6, LEUCO 8100, PLAQ 80K, INR 1,416/07: HB 12,9 HT 39,5 LEU 7800 PLT 156000 CR 1,6 PCR 13,38 K 5,2 CaI 1,07 MG 2,6 NA 130 U 132\u200b08/2023- ECO: AE: 44/ FE: 50%/ RITMO REGULAR/ SEM DEMAIS ALTERAÇOES\u200b08/2023- ausencia de sinais de tvp \u200bMEDICAÇÃO EM USO:-PANTOPRAZOL 40MG CEDO-CONCOR 5MG 1/2 CP CEDO-CLOPIDOGREL 75MG 1X/DIA-ATORVASTATINA 40MG 1X/DIA-COMBIRON FOLICO 1CP DIAS ALTERNADOS-ALENIA 12/400 2XDIA\u200b-COLCHICINA 0,5MG 1X/DIA\u200bSINAIS VITAIS:\u200b NÃO REALIZADO INDICADORES DE QUALIDADE: CUIDADO PALIATIVO: CID Z51.5: Paciente tem dor? ( )Sim ( )Não – Se sim, Nota 0 à 10 ( );\u200bAVC: CID I64: O paciente tem sonda vesical de demora? ( )Sim ( )Não ;S. DEMENCIAL: CID F03: Paciente já teve avaliação de fonoaudiologia?( )Sim ( )Não – Quando ( )3 a 6 meses ( )> 6 meses ;DIABETES MELLITUS: CID: E10: Hemoglobina Glicada? < 7 ( )Sim ( )Não – período de até 120 dias ( )Sim ( )Não CONDUTAS E AJUSTE DE PRESCRIÇÃO : \u200b-DIMINUO ATORVA 20MG \u200b-SUSPENDO COMBIRON \u200b-RETORNO EM DEZEMBRO, NO RETORNO TRAZER PACIENTE E FAZER TROCA DE GTT \u200b ATIVIDADES DA VIDA DIÁRIA (AVD) BANHO HIGIENIZAÇÃO MOBILIDADE Não recebe ajuda. É independente. Vai ao banheiro , limpa-se e arruma as roupas sem assistência (pode usar objetos de suporte como bengala, andador ou cadeira de rodas. Pode gerenciar fraldas noturnas, papagaio ou comadre, esvaziando – os pela manha Move-se para a cama ou para fora dela. Consegue sentar-se ou sair da cadeira de rodas sem assistência (pode usar objetos de suporte como bengala e andador). Necessita de assistência no banho apenas para uma parte do corpo(tal como as costas ou a perna) Recebe assistência para ir ao banheiro ,para limpar-se ou para arrumar as roupas após a eliminação. Recebe assistência no uso de fraldas, comadre ou papagaio. Move-se para a cama ou para fora dela com assistência. Necessita de assistência no banho Não vai a um local designado como “banheiro” para urinar ou defecar. Não sai da cama. VESTIR\u200b CONTINÊNCIA ALIMENTO Escolhe a roupa e veste-se completamente, sem assistência Controla urina e movimento de evacuação completamente. Alimenta-se sem assistência Escolhe a roupa e veste-se sem assistência, exceto para amarrar os sapatos.Tem “acidentes” ocasionais. . Alimenta-se , necessitando apenas de assistência para cortar carne ou espalhar manteiga de pão Recebe assistência para escolher a roupa ou para vestir-se ou não fica vestido, total ou parcialmenteNecessita supervisão para controle de micção e evacuação. Usa catéter ou é incontinente Necessita de assistência para alimentar-se parcial ou completamente. Usa sondas ou fluídos intravenosos. Resultado: I – Independente A- Com Assistência D - Dependente FOCO DO TELEFONE : CLASSIFICAÇÃO PÓS-CONSULTA - RETORNO A B C D E Homecare 26/09/2023 16:54:29 Dra. ALINE JORDAO COSTA (CRM 184738)CRM 184738 ____________________ _____________________________________ Data Assinatura do médico \u200b'],
    [' ANAMNESE - CENTRO CLÍNICO PACIENTE COM CEFALEIA E PA ELEVADA ROTINEIRAMENTEMAPA DA PA 24H - MEDIA 156X110 Sinais VitaisPA: 1160 X 120 FC:70 FR: 16 T: Dor: Exame Físico: NORMAL Hipótese Diagnóstica: HAS G 110, RESTANTE NORMAL Conduta: INICIO ANLODIPINA \u200b']
]

In [ ]:
spark_df = spark.createDataFrame(textos).toDF("text")

In [ ]:
result_df = nlpPipeline.fit(spark_df).transform(spark_df)
exploded_df = result_df.select(
    F.explode(
        F.arrays_zip(
            F.expr("ner_chunk_simplificado.result as chunk_simplificado"),
            F.expr("ner_chunk_simplificado.metadata as metadata_simplificado"),
            F.expr("ner_chunk_detalhe.result as chunk_detalhe"),
            F.expr("ner_chunk_detalhe.metadata as metadata_detalhe"),
        )
    ).alias("cols")
).select(
    F.expr("cols.chunk_simplificado"),
    F.expr("cols.metadata_simplificado.entity").alias("entidade_simplificado"),
    F.expr("cols.chunk_detalhe"),
    F.expr("cols.metadata_detalhe.entity").alias("entidade_detalhe")
).toPandas()

exploded_df.head()

,chunk_simplificado,entidade_simplificado,chunk_detalhe,entidade_detalhe
0,05/09/2023,DATE,6,AGE
1,Marcelo Fiusa,NAME,05/09/2023,DATE
2,84,AGE,Marcelo Fiusa,DOCTOR
3,98,AGE,60,AGE
4,20,AGE,84,AGE


In [ ]:
exploded_df.shape

(25, 4)

In [ ]:
exploded_df

,chunk_simplificado,entidade_simplificado,chunk_detalhe,entidade_detalhe
0,05/09/2023,DATE,6,AGE
1,Marcelo Fiusa,NAME,05/09/2023,DATE
2,84,AGE,Marcelo Fiusa,DOCTOR
3,98,AGE,60,AGE
4,20,AGE,84,AGE
5,55,AGE,98,AGE
6,SANTOS,NAME,20,AGE
7,PAULA,NAME,55,AGE
8,None,None,SANTOS,DOCTOR
9,None,None,PAULA,PATIENT


Usando o pipeline:

In [ ]:
deid_res = deid_pipeline.annotate([x[0] for x in textos])
deid_res[0]

{'masked': ['ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNIA INGUINAL Conduta:EXAMES \u200b'],
 'obfuscated': ['ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNIA INGUINAL Conduta:EXAMES \u200b'],
 'ner_chunk': [],
 'masked_fixed_length_chars': ['ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNIA INGUINAL Conduta:EXAMES \u200b'],
 'sentence': ['ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNIA INGUINAL Condut

In [ ]:
deid_res[0].keys()

dict_keys(['masked', 'obfuscated', 'ner_chunk', 'masked_fixed_length_chars', 'sentence', 'masked_with_chars'])

In [ ]:
sentences = []
masked = []
masked_with_chars = []
masked_fixed_length_chars = []
obfuscated = []

for res in deid_res:
    sentences.append(res["sentence"])
    masked.append(res["masked"])
    masked_with_chars.append(res["masked_with_chars"])
    masked_fixed_length_chars.append(res["masked_fixed_length_chars"])
    obfuscated.append(res["obfuscated"])


df = pd.DataFrame(
    list(zip(
        sentences,
        masked,
        masked_with_chars,
        masked_fixed_length_chars,
        obfuscated
    )),
    columns= ["Frase", "Anonimizado com Entidade", "Anonimizado com Caractere", "Anonimizado com Número Fixo de Caractere", "Anonimizado com substituição de Nomes"]
)
df

,Frase,Anonimizado com Entidade,Anonimizado com Caractere,Anonimizado com Número Fixo de Caractere,Anonimizado com substituição de Nomes
0,[ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNI...,[ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNI...,[ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNI...,[ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNI...,[ANAMNESE - CENTRO CLÍNICO Queixa Principal: DOR INGUINOESCROTAL ESQ A/E Alergias: Nega Alergia à Medicamentos HDA: HPP: Sinais VitaisPA: X FC: FR: T: Dor: Exame Físico: Hipótese Diagnóstica:HERNI...
1,[EVOLUÇÃO DE ENFERMAGEM 6 º DIH Hipótese Diagnóstica: K35 Apendicite aguda Evolução (estado geral): 05/09/2023 09:30 Faço contato com Dr Marcelo Fiusa via telefone informando que paciente esta com...,[EVOLUÇÃO DE ENFERMAGEM <AGE> º DIH Hipótese Diagnóstica: K35 Apendicite aguda Evolução (estado geral): <DATE> 09:30 Faço contato com Dr <DOCTOR> via telefone informando que <SEX> esta com cefalei...,[EVOLUÇÃO DE ENFERMAGEM * º DIH Hipótese Diagnóstica: K35 Apendicite aguda Evolução (estado geral): [********] 09:30 Faço contato com Dr [***********] via telefone informando que [******] esta com...,[EVOLUÇÃO DE ENFERMAGEM **** º DIH Hipótese Diagnóstica: K35 Apendicite aguda Evolução (estado geral): **** 09:30 Faço contato com Dr **** via telefone informando que **** esta com cefaleia intens...,[EVOLUÇÃO DE ENFERMAGEM 4 º DIH Hipótese Diagnóstica: K35 Apendicite aguda Evolução (estado geral): 05/09/2023 09:30 Faço contato com Dr Vasco Soares via telefone informando que homen esta com cef...
2,"[​PSA​​PACIENTE REFERE LESÃO PRURIGINOSA HA 2 DIAS EM DORSO, AXILA E REGIAO EPIGASTRICA, HOJE ASSOCIADAS A EDEMA PERIORBITAL., ​​EXAME FISICO​BEG CHAAAE LOTE ​NEURO: ECG 15, PIFR ​PRESENÇA DE LES...","[​PSA​​PACIENTE REFERE LESÃO PRURIGINOSA HA 2 DIAS EM DORSO, AXILA E REGIAO EPIGASTRICA, HOJE ASSOCIADAS A EDEMA PERIORBITAL., ​​EXAME FISICO​BEG CHAAAE LOTE ​NEURO: ECG 15, PIFR ​PRESENÇA DE LES...","[​PSA​​PACIENTE REFERE LESÃO PRURIGINOSA HA 2 DIAS EM DORSO, AXILA E REGIAO EPIGASTRICA, HOJE ASSOCIADAS A EDEMA PERIORBITAL., ​​EXAME FISICO​BEG CHAAAE LOTE ​NEURO: ECG 15, PIFR ​PRESENÇA DE LES...","[​PSA​​PACIENTE REFERE LESÃO PRURIGINOSA HA 2 DIAS EM DORSO, AXILA E REGIAO EPIGASTRICA, HOJE ASSOCIADAS A EDEMA PERIORBITAL., ​​EXAME FISICO​BEG CHAAAE LOTE ​NEURO: ECG 15, PIFR ​PRESENÇA DE LES...","[​PSA​​PACIENTE REFERE LESÃO PRURIGINOSA HA 2 DIAS EM DORSO, AXILA E REGIAO EPIGASTRICA, HOJE ASSOCIADAS A EDEMA PERIORBITAL., ​​EXAME FISICO​BEG CHAAAE LOTE ​NEURO: ECG 15, PIFR ​PRESENÇA DE LES..."
3,[​​CONSULTA MÉDICA CASE (CMR)​​ Retorno Repouso​​DIAGNÓSTICOS: ​-SINDROME DEMENCIAL PROVAVEL VASCULAR CDR 2​>>GTT 12/2022-TROCA EM 07/2023​>>LPP SACRAL GRAU 3- CICATRIZADA​-O2 ESPORADICO POS BANHO...,[​​CONSULTA MÉDICA CASE (CMR)​​ Retorno Repouso​​DIAGNÓSTICOS: ​-SINDROME DEMENCIAL PROVAVEL VASCULAR CDR 2​>>GTT <DATE> EM 07/2023​>>LPP SACRAL GRAU 3- CICATRIZADA​-O2 ESPORADICO POS BANHO-DAC CO...,[​​CONSULTA MÉDICA CASE (CMR)​​ Retorno Repouso​​DIAGNÓSTICOS: ​-SINDROME DEMENCIAL PROVAVEL VASCULAR CDR 2​>>GTT [***********] EM 07/2023​>>LPP SACRAL GRAU 3- CICATRIZADA​-O2 ESPORADICO POS BANHO...,[​​CONSULTA MÉDICA CASE (CMR)​​ Retorno Repouso​​DIAGNÓSTICOS: ​-SINDROME DEMENCIAL PROVAVEL VASCULAR CDR 2​>>GTT **** EM 07/2023​>>LPP SACRAL GRAU 3- CICATRIZADA​-O2 ESPORADICO POS BANHO-DAC COM ...,[​​CONSULTA MÉDICA CASE (CMR)​​ Retorno Repouso​​D